In [2]:
import os
import requests
import csv
from time import sleep

In [12]:
import requests
import time
import json

base_url = "https://dackkms.gov.in/Account/API/kKMS_QueryData.aspx"

def test_api(state, district, month, year):
    state_code = f"{state:02d}"  # Changed back to 2 digits
    district_code = f"{state_code}{district:02d}"
    params = {
        "StateCD": state_code,
        "DistrictCd": district_code,
        "Month": month,
        "Year": year
    }
    try:
        response = requests.get(base_url, params=params)
        return response.status_code, response.json()
    except requests.RequestException as e:
        return 0, str(e)

def extract_info(content):
    if content['ResponseCode'] == '1' and content['Response'] == 'Data Found':
        if content['data']:
            return content['data'][0]['StateName'], content['data'][0]['DistrictName']
    return None, None

def main():
    states = {}
    districts = {}
    total_requests = 0
    successful_extractions = 0

    print("Starting the discovery process...")

    for state in range(1, 40):  # Changed to check state codes from 01 to 99
        state_code = f"{state:02d}"
        districts[state_code] = {}
        print(f"\nChecking state code: {state_code}")

        for district in range(1, 100):  # Assuming max 99 districts per state
            total_requests += 1
            status, content = test_api(state, district, 1, 2023)  # Using January 2023
            
            if status == 200 and isinstance(content, dict):
                state_name, district_name = extract_info(content)
                
                if state_name and district_name:
                    successful_extractions += 1
                    if state_code not in states:
                        states[state_code] = state_name
                        print(f"Discovered new state: {state_name} ({state_code})")

                    district_code = f"{state_code}{district:02d}"
                    districts[state_code][district_code] = district_name

                    print(f"Found: State {state_name} ({state_code}), District {district_name} ({district_code})")
                else:
                    print(f"No data for state code {state_code}, district number {district:02d}")
            else:
                print(f"Failed or empty response for state code {state_code}, district number {district:02d}")

            time.sleep(0.5)  # Add a small delay to avoid overwhelming the server

            # Print progress every 10 requests
            if total_requests % 10 == 0:
                print(f"Progress: {total_requests} requests made, {successful_extractions} successful extractions")

        # Add a 15-second break after processing each state
        print(f"Finished processing state code {state_code}. Taking a 15-second break...")
        time.sleep(15)

    # Save the results
    with open('states.json', 'w') as f:
        json.dump(states, f, indent=2)
    print("\nSaved states data to states.json")

    with open('districts.json', 'w') as f:
        json.dump(districts, f, indent=2)
    print("Saved districts data to districts.json")

    print(f"\nDiscovery process completed.")
    print(f"Total requests made: {total_requests}")
    print(f"Successful extractions: {successful_extractions}")
    print(f"States discovered: {len(states)}")
    print(f"Districts discovered: {sum(len(districts[state]) for state in districts)}")

if __name__ == "__main__":
    main()

Starting the discovery process...

Checking state code: 01
Discovered new state: ANDHRA PRADESH (01)
Found: State ANDHRA PRADESH (01), District SRIKAKULAM (0101)
Found: State ANDHRA PRADESH (01), District VIZIANAGARM (0102)
Found: State ANDHRA PRADESH (01), District VISAKHAPATNAM (0103)
Found: State ANDHRA PRADESH (01), District EAST GODAVARI (0104)
Found: State ANDHRA PRADESH (01), District WEST GODAVARI (0105)
Found: State ANDHRA PRADESH (01), District KRISHNA (0106)
Found: State ANDHRA PRADESH (01), District GUNTUR (0107)
Found: State ANDHRA PRADESH (01), District PRAKASAM (0108)
Found: State ANDHRA PRADESH (01), District NELLORE (0109)
Found: State ANDHRA PRADESH (01), District KURNOOL (0110)
Progress: 10 requests made, 10 successful extractions
Found: State ANDHRA PRADESH (01), District ANANTPUR (0111)
Found: State ANDHRA PRADESH (01), District Y S R  (0112)
Found: State ANDHRA PRADESH (01), District CHITTOOR (0113)
No data for state code 01, district number 14
No data for state c

KeyboardInterrupt: 